<a href="https://colab.research.google.com/github/MoChiaoKai/Kai0214/blob/main/EX0406_ET_Today_%E6%96%B0%E8%81%9E%E6%A8%99%E9%A1%8C%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-ettoday_news.csv'
df = pd.read_csv(url)
df.head()


,類別,標題
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   類別      29216 non-null  object
 1   標題      29216 non-null  object
dtypes: object(2)
memory usage: 456.6+ KB


In [6]:
!wget -O dict.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
!wget -O stopwords.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt

--2025-06-11 07:35:56--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4154480 (4.0M) [application/octet-stream]
Saving to: ‘dict.txt’

dict.txt            100%[===================>]   3.96M  --.-KB/s    in 0.05s   

2025-06-11 07:35:56 (73.7 MB/s) - ‘dict.txt’ saved [4154480/4154480]

--2025-06-11 07:35:56--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8129 (7.9

In [7]:
import jieba
jieba.set_dictionary('dict.txt')

def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return stopwords

def jieba_cut_with_stopwords(text, stopwords):
    words = jieba.cut(text)
    return [word for word in words if word not in stopwords and word.strip() != '']

stopwords = load_stopwords('stopwords.txt')
df['jieba'] = df['標題'].apply(lambda x: ' '.join(jieba_cut_with_stopwords(x, stopwords)) )

Building prefix dict from /content/dict.txt ...
DEBUG:jieba:Building prefix dict from /content/dict.txt ...
Dumping model to file cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
Loading model cost 0.527 seconds.
DEBUG:jieba:Loading model cost 0.527 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [8]:
df.head()


,類別,標題,jieba
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利,青年 座談 AI 取代 關鍵 力 葉丙成 這兩種 人才 真正 把握 時間 紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮,蕭美琴 直播 體驗 曝 咪琴 嚴選 邀 網友 挺 花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論,疑似 國軍 美 受訓 曝光 陸軍 評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬,蔡英文 柯文哲 有談 NCC 府 沒談 人選 政黨 比例 私下 喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊,總統府 否認 蔡英文 柯喬 NCC 委員 黃國昌 嗆 醜聞 揭露 還敢 說謊


In [11]:
from sklearn.model_selection import train_test_split
X = df['jieba']
y = df['類別']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [14]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [16]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.score(X_train, y_train)


0.9073677905185692

In [17]:
from sklearn.metrics import accuracy_score
y_pred = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7838809034907598


In [18]:
text = '很棒！新聞寫得很完整，記者真是辛苦了，幫你拍拍手加油打氣！'
c = vectorizer.transform([' '.join(jieba_cut_with_stopwords(text, stopwords))])
c = nb.predict(c)[0]
category = label_encoder.classes_[c]
print(category)


9
